# Introduction

In this notebook, we'll learn how EMERGENT models parameter spaces and finds extrema. First, we'll study how EMERGENT chooses experimental data points to acquire, governed by the sampling method chosen by you, the user. Let's look at the base Sampling class:

In [3]:
from emergent.samplers.sampling import Sampling
Sampling??

In order to define a sampling method, you should override the \_run() method, as we will look at shortly. For now, understand that this method simply executes a bunch of experiments and returns the points (parameter space coordinates) and costs (experimental results). After the data is acquired, the \_finish() method allows a model to be fit to the data, then goes to either the first, last, or best point sampled. The Sampling class also implements a standardized plot() method which returns a handle to a 2D plot of the parameter space (currently only 2D spaces are supported).

Now let's look at how to define a sampling method. Consider the Grid sampling class:

In [4]:
from emergent.samplers import Grid
Grid??

As we can see, defining new sampling methods is easy - just override the \_run() method, then define any parameters you don't want to hard-code; these parameters will automatically appear in the GUI to be set before starting a run. In this case, the \_run() method creates a uniform grid between 0 and 1 in _N_ dimensions, runs the experiment at each point, and returns the results.

<div class="alert alert-block alert-warning">
Don't worry about the actual ranges - the sampling takes places in a normalized subspace from 0 to 1 along all coordinates, and EMERGENT automatically scales to the ranges defined by the Hub.settings dictionary, also displayed in the GUI's network tree.
</div>


Grid sampling can be useful for exploring experiments with 1 or 2 knobs, but the number of iterations required grows exponentially in the number of dimensions. For experiments with many free parameters, we can sample more intelligently with the _online_ sampling method:

In [5]:
from emergent.samplers import Online
Online??

The basic idea is that we attach a model to the sampling process - this could be based on physical intuition or just a general modeler like a Gaussian process. The sampling proceeds as follows:

1. Pre-sample some number of randomly generated points.
2. Fit the model to all acquired data.
3. Numerically minimize the _effective cost_ over the modeled surface to choose the next point.
4. Repeat step 3 for each point in a batch, then return to step 2, and repeat.

The _effective cost_ is the magic bullet that makes this work so well. Instead of sampling uniformly, we sample intelligently, using all of the knowledge gathered about the experiment to suggest the next point. But what function should we use for the effective cost? A good first guess is the amplitude of the parameter surface itself, i.e. seeking extrema where the signal-to-noise is high. However, if your model can also generate an uncertainty estimate for a point, then it works well to optimize some linear combination of the amplitude and the uncertainty. By iterating through various combinations, we can go from optimizer mode (seeking low amplitude) and explorer mode (seeking high uncertainty), refining the model very quickly.

Let's now examine how models are implemented in EMERGENT, starting with the base Model class:

In [6]:
from emergent.models.model import Model
Model??

When defining a model, you should override the fit() and predict() methods. The former should fit the model based on all acquired data so far, while the latter should generate a prediction of the experimental result (and possibly the uncertainty) at a given state dictionary. The Model class implements a number of other functions too, including generation of the next_sample() in terms of the explorer/optimizer tradeoff _b_ and the ability to plot() the modeled parameter surface in 2D.

Now let's look at a specific implementation of a model, Gaussian process regression:

In [7]:
from emergent.models import GaussianProcess
GaussianProcess??

Just as with defining a Sampling subclass, it is very simple to define a new model. In this case, we initialize a regressor in the __init__ method, then override the Model.fit() and Model.predict() methods with the specific syntax required for the GaussianProcessRegressor class imported from scikit-learn.